<a href="https://colab.research.google.com/github/ShriDurga17/Text-Summarizer/blob/main/ClusteringFMLMINI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/article val data set.csv')

In [ ]:
df.head()

,id,Article,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232
0,0e99444b1d4e7b8886582e371a6ce939,Pak PM Imran Khan expresses solidarity with In...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adfa04f5e1e7403dd90f5cddd29b6c37,"The Pentagon on Monday, June 18, 2018, formall...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,77d978318d3503fc6485c138ae3f3aa2,Taliban fighters patrol in the Wazir Akbar Kha...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,973c4f387dc9f48e1be58608101bc3d6,A top official of American defence intelligen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cb502ddb99c6a53d46b300027b4fe589,Boris Johnson's advisor under pressure over lo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
text=df["Article"][0]

In [ ]:
text

'Pak PM Imran Khan expresses solidarity with India over COVID-19 crisisPakistan Prime Minister Imran Khan on Saturday expressed his solidarity with the people of India amid the "dangerous" second wave of COVID-19, adding that "we must fight this global challenge confronting humanity together"."I want to express our solidarity with people of India as they battle a dangerous wave of #COVID19. Our prayers for speedy recovery go to all those suffering from pandemic in our neighbourhood and world," tweeted Mr Khan."We must fight this global challenge confronting humanity together," he added.Pakistan Foreign Minister Shah Mahmood Qureshi also expressed support to the people of India and said the COVID-19 crisis is yet another reminder that humanitarian issues require responses beyond political consideration."We express our support to the people of India in the wake of the current wave of #COVID19 infections that has hit our region hard. On behalf of the people of Pakistan, I extend our heart

In [ ]:
import nltk
nltk.download('punkt')   # one time execution
from nltk.tokenize import sent_tokenize
sentence = sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import re
nltk.download('stopwords')  # one time execution
from nltk.corpus import stopwords
corpus = []
for i in range(len(sentence)):
    sen = re.sub('[^a-zA-Z]', " ", sentence[i])
    sen = sen.lower()
    sen = sen.split()
    sen = ' '.join([i for i in sen if i not in stopwords.words('english')])
    corpus.append(sen)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from gensim.models import Word2Vec
all_words = [i.split() for i in corpus]
model = Word2Vec(all_words, min_count=1,size= 300)

In [ ]:
sent_vector=[]
for i in corpus:

    plus=0
    for j in i.split():
        plus+= model.wv[j]
    plus = plus/len(i.split())

    sent_vector.append(plus)

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
n_clusters = 5
kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(sent_vector)

In [ ]:
t1=[]
from scipy.spatial import distance
my_list=[]
for i in range(n_clusters):
    my_dict={}

    for j in range(len(y_kmeans)):

        if y_kmeans[j]==i:
            my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))


for i in sorted(my_list):
    print(sentence[i])
    t1.append(sentence[i])

Pak PM Imran Khan expresses solidarity with India over COVID-19 crisisPakistan Prime Minister Imran Khan on Saturday expressed his solidarity with the people of India amid the "dangerous" second wave of COVID-19, adding that "we must fight this global challenge confronting humanity together".
Our prayers for speedy recovery go to all those suffering from pandemic in our neighbourhood and world," tweeted Mr Khan.
"We express our support to the people of India in the wake of the current wave of #COVID19 infections that has hit our region hard.
On behalf of the people of Pakistan, I extend our heartfelt sympathies to the affected families in #India," Mr Qureshi tweeted.
"#COVID19 is yet another reminder that humanitarian issues require responses beyond political consideration.


In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge
ROUGE = Rouge()


In [ ]:
def fetch_fscores(prediction, reference):
    rouge_res = ROUGE.get_scores(t1,text)[0]
    fscores = [rouge_res["rouge-1"]["f"], rouge_res["rouge-2"]["f"], rouge_res["rouge-l"]["f"]]
    return fscores

In [ ]:
t1

['Pak PM Imran Khan expresses solidarity with India over COVID-19 crisisPakistan Prime Minister Imran Khan on Saturday expressed his solidarity with the people of India amid the "dangerous" second wave of COVID-19, adding that "we must fight this global challenge confronting humanity together".',
 'Our prayers for speedy recovery go to all those suffering from pandemic in our neighbourhood and world," tweeted Mr Khan.',
 '"We express our support to the people of India in the wake of the current wave of #COVID19 infections that has hit our region hard.',
 'On behalf of the people of Pakistan, I extend our heartfelt sympathies to the affected families in #India," Mr Qureshi tweeted.',
 '"#COVID19 is yet another reminder that humanitarian issues require responses beyond political consideration.']

In [ ]:
t1 = ' '.join(t1)
print(t1)

Pak PM Imran Khan expresses solidarity with India over COVID-19 crisisPakistan Prime Minister Imran Khan on Saturday expressed his solidarity with the people of India amid the "dangerous" second wave of COVID-19, adding that "we must fight this global challenge confronting humanity together". Our prayers for speedy recovery go to all those suffering from pandemic in our neighbourhood and world," tweeted Mr Khan. "We express our support to the people of India in the wake of the current wave of #COVID19 infections that has hit our region hard. On behalf of the people of Pakistan, I extend our heartfelt sympathies to the affected families in #India," Mr Qureshi tweeted. "#COVID19 is yet another reminder that humanitarian issues require responses beyond political consideration.


In [ ]:
fetch_fscores(t1,text)

[0.6872586827477231, 0.6232294574449679, 0.6872586827477231]